In [ ]:
import pandas as pd
import numpy as np
import torch

In [2]:
df6=pd.read_csv('/content/drive/MyDrive/mi/realv11.csv',index_col=0)

In [3]:
# label 값이 0, 1, 2, 3인 행을 각각 1/4씩 추출하여 새로운 데이터프레임 생성
df6_filtered = df6.groupby('labels').apply(lambda x: x.sample(frac=0.25)).reset_index(drop=True) #각 군집별로 0.25추출
df6_filtered = df6_filtered.sample(frac=1, random_state=42).reset_index(drop=True)

# 'labels' 컬럼 제거
#df6_filtered = df6_filtered.drop(columns=['labels'])
# "BUY1_MKT" 문자열을 포함하는 컬럼을 찾아서 삭제
df6_filtered = df6_filtered[df6_filtered.columns.drop(list(df6_filtered.filter(like='BUY1_MKT')))]


# 결과 확인
df6_filtered.to_csv('/content/drive/MyDrive/mi/realv13.csv',encoding='utf-8-sig')



In [1]:

from sklearn.metrics.pairwise import cosine_similarity


# 데이터 로드 및 전처리
df = pd.read_csv('/content/drive/MyDrive/mi/realv13.csv',index_col=0)
df = df.drop(columns=['labels'])
df=df.T #transpose
df = df.replace('*', 0) #결측치 0 처리

new_data = [] #횟수 세기
for column in df.columns:
    value_counts = df[column].value_counts()
    new_data.append(value_counts)
new_df = pd.DataFrame(new_data)
new_df = new_df.fillna(0).astype(int)
new_df.index = df.columns

# Example DataFrame creation (random values, 11 columns, 1000 rows)
num_columns = 11 #사용 총 column
num_rows = 200827 #총 row
data = np.random.randint(0, 11, size=(num_rows, num_columns))
desired_columns = [
    "Industrials", "Health Care", "Information Technology",
    "Communication Services", "Consumer Staples",
    "Consumer Discretionary",  "Materials",
    "Finance", "Utilities", "Energy",'RealEstate' #총 11개의 섹터 사용
]
df = new_df[desired_columns]


invested_industry_distribution = df.sum() / df.sum().sum()

# Calculate cosine similarity for each investment compared to invested distribution
similarity_scores = cosine_similarity(df, invested_industry_distribution.values.reshape(1, -1)) #코사인 유사도 사용

# Create a Series of similarity scores indexed by investment
similarity_series = pd.Series(similarity_scores.flatten(), index=df.index)

# Sort investments by similarity score (higher scores are more similar)
recommended_investments = similarity_series.sort_values(ascending=False)

print("Invested industry distribution:\n", invested_industry_distribution)
print("\nRecommended investments:\n", recommended_investments)

recommended_investments  #간단히 확인


Invested industry distribution:
 Industrials               0.156294
Health Care               0.106363
Information Technology    0.318453
Communication Services    0.121699
Consumer Staples          0.020885
Consumer Discretionary    0.092175
Materials                 0.104957
Finance                   0.033059
Utilities                 0.008556
Energy                    0.034668
RealEstate                0.002891
dtype: float64

Recommended investments:
 92600     0.983857
68088     0.983857
140706    0.983857
136650    0.983857
71549     0.983857
            ...   
58459     0.000000
137571    0.000000
137570    0.000000
137568    0.000000
200826    0.000000
Length: 200827, dtype: float64


92600     0.983857
68088     0.983857
140706    0.983857
136650    0.983857
71549     0.983857
            ...   
58459     0.000000
137571    0.000000
137570    0.000000
137568    0.000000
200826    0.000000
Length: 200827, dtype: float64

In [2]:
df2=pd.read_csv('/content/drive/MyDrive/mi/realv13.csv',index_col=0)
df2 = df2.drop(columns=['labels'])
df2 = df2.replace('*', '0')
df2=df2.T

In [3]:
foreign=pd.read_csv('/content/drive/MyDrive/mi/조롱이/해외3.csv')
kor=pd.read_csv('/content/drive/MyDrive/mi/조롱이/kor2.csv')
kor['종목코드'] = kor['종목코드'].apply(lambda x: str(x) if 'K' in str(x) or 'L' in str(x) else int(x))
kor['종목코드'] = kor['종목코드'].astype(str)

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
# Extract the invested industry distribution for the first row (index 1)
n=15   # n번째 행 (사람)

target_distribution = df.iloc[n,:].values.reshape(1, -1)

# Calculate cosine similarity for the first row compared to other rows
similarity_scores = cosine_similarity(df, target_distribution)

# Create a Series of similarity scores indexed by investment
similarity_series = pd.Series(similarity_scores.flatten(), index=df.index)

# Sort investments by similarity score (higher scores are more similar)
top_similar_investments = similarity_series.sort_values(ascending=False).head(50) #top50 사용

# Extract the indices (investments) from the top_similar_investments Series
similar_investments_indices = top_similar_investments.index.tolist()
similar_investments_indices = [int(index) for index in similar_investments_indices]
similar_investments_indices =[index - 1 for index in similar_investments_indices]



df2=pd.read_csv('/content/drive/MyDrive/mi/realv13.csv',index_col=0)
df2 = df2.drop(columns=['labels'])
df2 = df2.replace('*', '0')
df2=df2.T

df3 = df2.iloc[0:12, :]


# Create an empty DataFrame to store the results
top_items_df = pd.DataFrame()

excluded_items = ['0', 0, '0', "코스닥", "코스피", "나스닥", "뉴욕", "홍콩", "아멕스", "동경", "상해A", "상천A", "호치민"]

# Sort similar_investments_indices in ascending order
sorted_indices = sorted(similar_investments_indices)

# Iterate through the sorted indices
for index in sorted_indices:
    # Check if the index is within the valid range of column indices
    if index >= 0 and index < len(df3.columns):
        # Extract the column name
        column_name = df3.columns[index]

        # Exclude items in the excluded_items list
        if column_name not in excluded_items:
            # Extract the values in the column
            column_values = df3.iloc[:, index]

            # Count the occurrences of each item in the column
            item_counts = pd.Series(column_values).value_counts()

            # Apply the excluded_items filter
            filtered_item_counts = item_counts[~item_counts.index.isin(excluded_items)]

            # Append the top 10 items to the results DataFrame
            top_items = filtered_item_counts.head(50).reset_index()
            top_items.columns = ['Item', 'Count']
            top_items['Column'] = column_name  # Add the column name
            top_items_df = top_items_df.append(top_items)

# Reset the index of the results DataFrame
top_items_df = top_items_df.reset_index(drop=True)

# Sort top_items_df by Count column in descending order and then by similar_investments_indices
top_items_df = top_items_df.sort_values(by=['Count', 'Column'], ascending=[False, False])

# Print the top 10 items for each column (after excluding items)
print("Top 10 items for each column (after excluding items and sorted by Count and similar_investments_indices):")
top_items_df


Top 10 items for each column (after excluding items and sorted by Count and similar_investments_indices):


<ipython-input-4-861209edace6>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append(top_items)
<ipython-input-4-861209edace6>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append(top_items)
<ipython-input-4-861209edace6>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append(top_items)
<ipython-input-4-861209edace6>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  top_items_df = top_items_df.append(top_items)
<ipython-input-4-861209edace6>:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

,Item,Count,Column
64,51910,8,95886
14,47050,8,9539
39,TQQQ,7,43234
107,5935,5,197117
76,SOXL,5,111601
...,...,...,...
12,34220,1,5005
13,103140,1,5005
2,69620,1,4995
3,5930,1,4995


In [5]:
foreign2=pd.read_csv('/content/drive/MyDrive/mi/조롱이/해외2.csv')
kor2=pd.read_csv('/content/drive/MyDrive/mi/조롱이/kor.csv')
etf=pd.read_csv('/content/drive/MyDrive/mi/조롱이/etf.csv')

In [6]:
# Create a new column 'nation' with default value 'X'
top_items_df['nation'] = 'X' #기본으로 X 설정. 거래중지 구분.

# Iterate through rows in top_items_df
for index, row in top_items_df.iterrows():
    # Get the 'Item' value from the current row
    item = row['Item']

    # Check if 'item' exists in 'kor' DataFrame's '종목코드' column
    if item in etf['Symbol'].values:
        # If it exists in 'etf', set the 'nation' column to 'etf'
        top_items_df.at[index, 'nation'] = 'etf'
    elif item in kor2['종목코드'].values:
        # If it exists in 'kor', set the 'nation' column to '국내'
        top_items_df.at[index, 'nation'] = '국내'
    elif item in foreign2['Symbol'].values:
        # If it exists in 'foreign', set the 'nation' column to '해외'
        top_items_df.at[index, 'nation'] = '해외'


# Print the updated top_items_df
print("Updated top_items_df:")
top_items_df #국내 국외주식 및 etf 구별


Updated top_items_df:


,Item,Count,Column,nation
64,51910,8,95886,국내
14,47050,8,9539,국내
39,TQQQ,7,43234,X
107,5935,5,197117,국내
76,SOXL,5,111601,etf
...,...,...,...,...
12,34220,1,5005,국내
13,103140,1,5005,국내
2,69620,1,4995,국내
3,5930,1,4995,국내


In [9]:
#중복제거
# Create a new column 'nation' with default value 'X'
top_items_df['nation'] = 'X'

# Iterate through rows in top_items_df
for index, row in top_items_df.iterrows():
    # Get the 'Item' value from the current row
    item = row['Item']

    # Check if 'item' exists in 'etf' DataFrame's 'Symbol' column
    if item in etf['Symbol'].values:
        # If it exists in 'etf', set the 'nation' column to 'etf'
        top_items_df.at[index, 'nation'] = 'etf'
    elif item in kor2['종목코드'].values:
        # If it exists in 'kor2', set the 'nation' column to '국내'
        top_items_df.at[index, 'nation'] = '국내'
    elif item in foreign2['Symbol'].values:
        # If it exists in 'foreign2', set the 'nation' column to '해외'
        top_items_df.at[index, 'nation'] = '해외'


# Create a dictionary to store the counts for each unique 'Item'
max_count_dict = {}

# Create a dictionary to store the Count values to be added for each 'Item'
to_add_dict = {}

# Iterate through rows in top_items_df
for index, row in top_items_df.iterrows():
    # Get the 'Item' value, Count, and Column from the current row
    item = row['Item']
    count = row['Count']
    column = row['Column']

    # Check if 'item' already exists in max_count_dict
    if item in max_count_dict:
        # If it exists, update the maximum Count value if the current row has a larger Count
        if count > max_count_dict[item]:
            max_count_dict[item] = count
            to_add_dict[item] = to_add_dict.get(item, 0) + max_count_dict[item] - count
        else:
            to_add_dict[item] = to_add_dict.get(item, 0) + max_count_dict[item]
    else:
        # If it doesn't exist, initialize the maximum Count value with the current Count
        max_count_dict[item] = count

# Iterate through rows in top_items_df again and update Count values
for index, row in top_items_df.iterrows():
    item = row['Item']
    count = row['Count']
    to_add = to_add_dict.get(item, 0)
    top_items_df.at[index, 'Count'] = count + to_add

# Remove duplicate rows based on 'Item' and keep rows with maximum 'Column' value
top_items_df = top_items_df.sort_values(by=['Item', 'Column']).drop_duplicates(subset=['Item'], keep='first')

# Print the updated top_items_df
print("Updated top_items_df:")
top_items_df #위와 동일 코드







Updated top_items_df:


,Item,Count,Column,nation
65,10120,2,95886,국내
27,101670,1,15343,국내
13,103140,1,5005,국내
24,10620,1,13101,국내
52,108860,2,57339,국내
...,...,...,...,...
58,SQQQ,1,57343,X
42,TMF,1,43234,X
39,TQQQ,14,43234,X
91,TSLA,2,139173,해외


In [10]:
# 'nation' 열이 '해외'인 항목들 중에서 top 5를 뽑습니다.
top_overseas_items = top_items_df[top_items_df['nation'] == '해외'].nlargest(5, 'Count')

# 'nation' 열이 '국내'인 항목들 중에서 top 5를 뽑습니다.
top_domestic_items = top_items_df[top_items_df['nation'] == '국내'].nlargest(5, 'Count')

# 'nation' 열이  'etf'인 항목들 중에서 top 5를 뽑습니다.
top_etf_items = top_items_df[top_items_df['nation'] == 'etf'].nlargest(5, 'Count')

print("Top 5 Overseas Items:")
print(top_overseas_items)

print("\nTop 5 Domestic Items:")
print(top_domestic_items)

print("\nTop 5 etf Items:")
print(top_etf_items)


Top 5 Overseas Items:
    Item  Count  Column nation
40  IONQ      2   43234     해외
91  TSLA      2  139173     해외
95  CDIO      1  157672     해외
94  IBIO      1  157672     해외
37    MO      1   35768     해외

Top 5 Domestic Items:
     Item  Count  Column nation
3    5930     25    4995     국내
11   5935     11    5005     국내
14  47050      8    9539     국내
64  51910      8   95886     국내
6   86790      5    5005     국내

Top 5 etf Items:
      Item  Count  Column nation
76    SOXL      5  111601    etf
0   364980      2    4995    etf
70  371460      1  100004    etf
72  394670      1  100004    etf


In [11]:
top_overseas_items['Name'] = ""

# Item 컬럼 값을 비교하여 Name 컬럼에 값을 저장
for index, row in top_overseas_items.iterrows():
    item = row['Item']
    matching_row = foreign2[foreign2['Symbol'] == item]
    if not matching_row.empty:
        name_value = matching_row.iloc[0]['Name']
        top_overseas_items.at[index, 'Name'] = name_value

# 결과 확인
top_overseas_items

,Item,Count,Column,nation,Name
40,IONQ,2,43234,해외,IONQ Inc
91,TSLA,2,139173,해외,Tesla Inc
95,CDIO,1,157672,해외,Cardio Diagnostics Holdings Inc
94,IBIO,1,157672,해외,Ibio Inc
37,MO,1,35768,해외,Altria Group Inc


In [12]:
top_domestic_items['Name'] = ""

# Item 컬럼 값을 비교하여 Name 컬럼에 값을 저장
for index, row in top_domestic_items.iterrows():
    item = row['Item']
    matching_row = kor2[kor2['종목코드'] == item]
    if not matching_row.empty:
        name_value = matching_row.iloc[0]['종목명']
        top_domestic_items.at[index, 'Name'] = name_value

# 결과 확인
top_domestic_items

,Item,Count,Column,nation,Name
3,5930,25,4995,국내,삼성전자
11,5935,11,5005,국내,삼성전자우
14,47050,8,9539,국내,포스코인터내셔널
64,51910,8,95886,국내,LG화학
6,86790,5,5005,국내,하나금융지주


In [13]:
top_etf_items['Name'] = ""

# Item 컬럼 값을 비교하여 Name 컬럼에 값을 저장
for index, row in top_etf_items.iterrows():
    item = row['Item']
    matching_row = etf[etf['Symbol'] == item]
    if not matching_row.empty:
        name_value = matching_row.iloc[0]['Name']
        top_etf_items.at[index, 'Name'] = name_value

# 결과 확인_최종 eft 추천 top 5 이내 Name 추출
top_etf_items

,Item,Count,Column,nation,Name
76,SOXL,5,111601,etf,Direxion Daily Semiconductor Bull 3x Shares
0,364980,2,4995,etf,TIGER KRX2차전지K-뉴딜
70,371460,1,100004,etf,TIGER 차이나전기차SOLACTIVE
72,394670,1,100004,etf,TIGER 글로벌리튬&2차전지SOLACTIVE(합성)


In [ ]:
new_investor_distribution = [0.05, 0.4, 0.3, 0.05, 0, 0, 0, 0, 0, 0.2]#example
#새로운 투자자의 투자 원하는 분포 등등

new_investor_df = pd.DataFrame([new_investor_distribution], columns=df.columns)

# Calculate cosine similarity between the new investor and existing investors
similarity_scores = cosine_similarity(new_investor_df, df)

# Create a Series of similarity scores indexed by existing investors
similarity_series = pd.Series(similarity_scores.flatten(), index=df.index)

# Sort existing investors by similarity score (higher scores are more similar)
recommended_investors = similarity_series.sort_values(ascending=False)

print("\nRecommended similar investors:\n", recommended_investors)

#이러면 몇번 투자자의 성향으로 추천 가도 됨.
#여기까지가 종목만을 활용


Recommended similar investors:
 164156    0.991489
83142     0.984136
88989     0.984136
103716    0.984136
94219     0.984136
            ...   
100128    0.000000
100126    0.000000
26268     0.000000
26270     0.000000
88453     0.000000
Length: 176905, dtype: float64
